In [35]:
import hashlib
import statistics
import os
from sage.all import *

def FZ13_Setup(q):
    pp =[]
    g = randint (1, q-1)
    pp.append(q)
    pp.append(g)
    return pp

# Keygen
def FZ13_RG(pp, N):
    q = pp[0]
    g = pp[1]
    tmp = N-1
    
    x = [randint (1,q-1) for i in range (0,N)]
    y = [power_mod(g,x[i],q) for i in range (0,N)]
    R = y
    
    return R, x, y 

#sign
def RS(pp, N, y, x, i, R, m):
    # # Step 1 (Simulation)
    q = pp[0]
    g = pp[1]
    sigma = []
    
    c = [0]*N
    t = [0]*N
    a = [0]*N
    b = [0]*N
    
    rtg = hashlib.sha256()
    rtg.update(m)
    rtg.update(str(R).encode())
    
    hash_val = Integer('0x' + rtg.hexdigest()) # r  = H(M||R)
    
    r = [0] * N
    ab = ""
    sum_c = 0
    
    for j in range (0, N):
        if j != i :
            c[j] = randint (1,q-1)
            t[j] = randint (1,q-1)
            a[j] = (power_mod(g,t[j],q) * power_mod(y[j],c[j],q)) %q
            b[j] = ((power_mod(hash_val, t[j], q) * power_mod(power_mod(hash_val, x[i], q), c[j], q)) % q)
            sum_c += c[j]

    #Step 2
        if j == i:
            r[i] = randint (1,q-1)
            a[j] = power_mod(g, r[i], q)
            b[j] = power_mod(hash_val, r[i], q) 

        ab += str(a[j])
        ab += str(b[j])

    #Step 3
    rtg.update(ab.encode())          
    c[i] = (Integer('0x' + rtg.hexdigest()) - sum_c) %q
    t[i] = (r[i] - c[i] * x[i]) % q

    #Step 4
    tau = power_mod(hash_val, x[i], q)
    sigma.append(tau)

    for j in range (0, N):
        sigma.append(c[j])
        sigma.append(t[j])
    return sigma

#Verify
def FZ13_RV(pp, N, y, x, R, m, sigma, i):
    q = pp[0]
    g = pp[1]

    #extract sigma
    ver_tau = sigma[0]
    ver_c = []
    ver_t = []
    for j in range(1, 2*N, 2):
        ver_c.append(sigma[j])
        ver_t.append(sigma[j+1]) 

    rtg = hashlib.sha256()
    rtg.update(m)
    rtg.update(str(R).encode())
    ver_hash_val = Integer('0x' + rtg.hexdigest()) # r  = H(M||R)
    tmp_str = ""
    ver_sum_c = 0
    satisfied = true
    
    for j in range (0, N):
        if i != j:
            tmp =  (power_mod(g, ver_t[j], q) * power_mod(y[j], ver_c[j], q))%q
            hash_val_1 = (power_mod(ver_hash_val, ver_t[j], q) * power_mod(ver_tau, ver_c[j],q)) %q
        else:
            tmp =  power_mod(g, (ver_t[j] + x[j]* ver_c[j]) % q, q)
            hash_val_1 = power_mod(ver_hash_val, (ver_t[j] + x[i]*ver_c[j]) % q,q)
            
        tmp_str += str(tmp)
        tmp_str += str(hash_val_1)
        ver_sum_c += ver_c[j]
    rtg.update(tmp_str.encode())
    vp = Integer('0x' + rtg.hexdigest()) # r  = H(M||R)

    if (ver_sum_c % q != vp % q):
        satisfied = false
    if (satisfied == false):
        print("Satisfied", satisfied)
        
def FZ13(list_n, q, file):
    #read file
    m = b''
    # file need to sign
  
    # file size
    file_size = os.stat(file)
    print(f'File size (MB): {file_size.st_size / (1024 * 1024)}')
    rtg = hashlib.sha256()
    BLOCK_SIZE = 65536

    with open(file, 'rb') as f: 
        fb = f.read(BLOCK_SIZE)
        m += fb
        while len(fb) > 0:
             fb = f.read(BLOCK_SIZE)
    
    avg_keygen_time_list = []
    avg_sign_time_list = []
    avg_verify_time_list = []
    #remove file before update new result
    try:
        os.remove("fz13.txt")
    except:
        print('No such file')
    
    for n in list_n:
        N = power(2,n)
        gen_time_list = []
        sign_time_list = []
        verify_time_list = []
        
        for j in range (0, 10): # run 10 times
            print("N =", N, " - Interval: ", j)
            # Set up ring
            ring_Z = IntegerRing()
            i = ring_Z(randint (0, N-1)) #signer
            
            # Start algo
            # FZ13_Setup
            pp = FZ13_Setup(q)
            
            # Key Gen
            t1 = cputime()
            R, x, y = FZ13_RG(pp, N)
            gen_time_list.append(cputime(t1))

            # Sign
            t2 = cputime()
            sigma = RS(pp, N, y, x, i, R, m)
            sign_time_list.append(cputime(t2))

            # Verify
            t3 = cputime()
            FZ13_RV(pp, N, y, x, R, m, sigma, i)
            verify_time_list.append(cputime(t3))

        avg_gen_time = statistics.mean(gen_time_list)
        avg_sign_time = statistics.mean(sign_time_list)
        avg_verify_time = statistics.mean(verify_time_list)
       
        f = open("fz13.txt", "a")
        s = str(N) + ',' +  str(avg_gen_time) + ',' + str(avg_sign_time) + ','+ str(avg_verify_time) + '\n'
        f.write(s)
        f.close()

        print("Avg gen time:", avg_gen_time)
        print("Avg sign time:", avg_sign_time)
        print("Avg verify time:", avg_verify_time)
        print("")

        avg_keygen_time_list.append(avg_gen_time)
        avg_sign_time_list.append(avg_sign_time)
        avg_verify_time_list.append(avg_verify_time)
      
    return avg_gen_time, avg_sign_time, avg_verify_time_list

# file = "Slide.pdf"
# list_n = [2,3,4]
# qbit = 4
# q = random_prime ( 2^qbit , proof=True , lbound=2^(qbit-1) )
# avg_verify_time_list,avg_sign_time, avg_verify_time_list = FZ13(list_n, q, file)
